# ChatSeekrFlow

> [Seekr](https://www.seekr.com/) provides AI-powered solutions for structured, explainable, and transparent AI interactions.

This notebook provides a quick overview for getting started with Seekr [chat models](/docs/concepts/chat_models). For detailed documentation of all `ChatSeekrFlow` features and configurations, head to the [API reference](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.seekrflow.ChatSeekrFlow.html).

## Overview

`ChatSeekrFlow` class wraps a chat model endpoint hosted on SeekrFlow, enabling seamless integration with LangChain applications.

### Integration Details

| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [ChatSeekrFlow](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.seekrflow.ChatSeekrFlow.html) | [seekrai](https://python.langchain.com/docs/integrations/providers/seekr/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/seekrai?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/seekrai?style=flat-square&label=%20) |

### Model Features

| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ | ❌ | ❌ | ❌ | ✅ | ❌ | ✅ | ❌ |

### Supported Methods
`ChatSeekrFlow` supports all methods of `ChatModel`, **except async APIs**.

### Endpoint Requirements

The serving endpoint `ChatSeekrFlow` wraps **must** have OpenAI-compatible chat input/output format. It can be used for:
1. **Fine-tuned Seekr models**
2. **Custom SeekrFlow models**
3. **RAG-enabled models using Seekr's retrieval system**

For async usage, please refer to `AsyncChatSeekrFlow` (coming soon).


## Installation

Ensure you have the necessary dependencies installed:

```bash
pip install seekrai langchain langchain-community
```

You must also have an API key from Seekr to authenticate requests.


## API Key Setup

You'll need to set your API key as an environment variable to authenticate requests.

Run the following in a cell:

```python
import getpass
import os

os.environ["SEEKR_API_KEY"] = getpass.getpass("Enter your Seekr API key:")
```

Or manually assign it before running queries:

```python
SEEKR_API_KEY = "your-api-key-here"
```


# Getting Started with ChatSeekrFlow in LangChain

This notebook covers how to use SeekrFlow as a chat model in LangChain.

## 🔧 Setup

SeekrFlow is available in `langchain-community`, so you just need:

```sh
pip install -U langchain


You'll also need a SeekrFlow API Key 

In [ ]:
import getpass
import os

os.environ["SEEKR_API_KEY"] = getpass.getpass("Enter your Seekr API key:")
SEEKR_API_KEY = os.environ["SEEKR_API_KEY"]

## Basic Model Invocation

In [9]:
from langchain_community.chat_models.seekrflow import ChatSeekrFlow
from seekrai import SeekrFlow 
from langchain.schema import HumanMessage

# Initialize the Seekr API Client
#seekr_client = SeekrFlow(api_key=os.getenv("SEEKR_API_KEY"))
seekr_client = SeekrFlow(api_key=SEEKR_API_KEY)

# Create the LangChain chat model
llm = ChatSeekrFlow(
    client=seekr_client,
    model_name="meta-llama/Meta-Llama-3-8B-Instruct"
)


# Run a simple chat interaction
response = llm.invoke([HumanMessage(content="Hello, Seekr!")])
print(response.content)


Hello there! It's great to meet you! I'm Seekr, your friendly AI companion. I'm here to help you with any questions, topics, or conversations you'd like to have. What's on your mind today?


In [11]:
def test_stream():
    """Test synchronous invocation in streaming mode."""
    print("\n🔹 Testing Sync `stream()` (Streaming)...")

    for chunk in llm.stream([HumanMessage(content="Write me a haiku.")]):
        print(chunk.content, end="", flush=True)

# ✅ Ensure streaming is enabled
llm = ChatSeekrFlow(
    client=seekr_client,
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    streaming=True  # ✅ Enable streaming
)

# ✅ Run sync streaming test
test_stream()



🔹 Testing Sync `stream()` (Streaming)...
Here is a haiku:

Snowflakes gently fall
Blanketing the landscape white
Winter's peaceful hush

## Handling Stop Tokens

In [12]:

# Using a stop token to truncate output
messages = [HumanMessage(content="Tell me a story about a dragon.")]
response = llm.invoke(messages, stop=[":"])

print(response.content)


Here's a story about a magnificent dragon


## Error Handling & Debugging

In [13]:
from langchain_community.chat_models.seekrflow import ChatSeekrFlow
from langchain.schema import HumanMessage

# Define a minimal mock SeekrFlow client
class MockSeekrClient:
    """Mock SeekrFlow API client that mimics the real API structure."""

    class MockChat:
        """Mock Chat object with a completions method."""
        class MockCompletions:
            """Mock Completions object with a create method."""
            def create(self, *args, **kwargs):
                return {"choices": [{"message": {"content": "Mock response"}}]}  # Mimic API response

        completions = MockCompletions()

    chat = MockChat()

def test_initialization_errors():
    """Test that invalid ChatSeekrFlow initializations raise expected errors."""

    test_cases = [
        {
            "name": "Missing Client",
            "args": {"client": None, "model_name": "seekrflow-model"},
            "expected_error": "SeekrFlow client cannot be None."
        },
        {
            "name": "Missing Model Name",
            "args": {"client": MockSeekrClient(), "model_name": ""},
            "expected_error": "A valid model name must be provided."
        }
    ]

    for test in test_cases:
        try:
            print(f"Running test: {test['name']}")
            faulty_llm = ChatSeekrFlow(**test["args"])

            # If no error is raised, fail the test
            print(f"❌ Test '{test['name']}' failed: No error was raised!")
        except Exception as e:
            error_msg = str(e)
            assert test["expected_error"] in error_msg, f"Unexpected error: {error_msg}"
            print(f"✅ Expected Error: {error_msg}")

# Run test
test_initialization_errors()


Running test: Missing Client
✅ Expected Error: SeekrFlow client cannot be None.
Running test: Missing Model Name
✅ Expected Error: A valid model name must be provided.


In [14]:
response = seekr_client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[{"role": "user", "content": "What's the weather in NYC?"}],
    functions=[
        {
            "name": "get_weather",
            "description": "Retrieve the current weather",
            "parameters": {
                "type": "object",
                "properties": {"location": {"type": "string"}}
            }
        }
    ]
)
print(response)


TypeError: ChatCompletions.create() got an unexpected keyword argument 'functions'